In [16]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pypsa

from pathlib import Path

In [18]:
outdir = Path.cwd() / 'example_outputs_otherzones'

fn = "network_s_eso_solved.nc"
datadir = '{}_{}'
root = Path.cwd().parent / "resources" / "live_data"

In [ ]:
gpd.read_file(root / datadir.format("2024-01-07", 4) / "regions_offshore_s.geojson").head()

,name,geometry
0,4956,"POLYGON ((2.35762 51.70601, 1.14430 51.65117, ..."
1,4971,"POLYGON ((0.73681 50.37510, 0.60333 50.32806, ..."
2,4972,"POLYGON ((1.01679 51.67229, 0.87065 51.42376, ..."
3,4974,"POLYGON ((0.83567 51.36428, 0.89916 51.35722, ..."
4,4975,"POLYGON ((1.14600 51.08327, 1.35584 51.12764, ..."


In [19]:
def fix_snaps(df, date, period):
    df.index = [pd.Timestamp(date) + (period - 1) * pd.Timedelta(f'30min')]
    return df

pnoms = pd.DataFrame()
p = pd.DataFrame()
psets = pd.DataFrame()
line_loads = pd.DataFrame()
link_loads = pd.DataFrame()
prices = pd.DataFrame()

date = '2024-03-07'

for period in range(1, 48):

    n = pypsa.Network(root / datadir.format(date, period) / fn)    
    time = pd.Timestamp(date) + (period - 1) * pd.Timedelta(f'30min')

    pnoms = pd.concat((pnoms, n.generators[["p_nom"]].rename(columns={"p_nom": time})), axis=1)
    p = pd.concat((p, fix_snaps(n.generators_t["p"], date, period)), axis=0)
    line_loads = pd.concat((line_loads, fix_snaps(n.lines_t["p0"], date, period)), axis=0)
    link_loads = pd.concat((link_loads, fix_snaps(n.links_t["p0"], date, period)), axis=0)
    prices = pd.concat((prices, fix_snaps(n.buses_t["marginal_price"], date, period)), axis=0)

    psets = pd.concat((psets, n.generators[["p_set"]].rename(columns={"p_set": time})), axis=1)

INFO:pypsa.io:Imported network network_s_eso_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_eso_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_eso_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_eso_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_eso_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_eso_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_eso_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_eso_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network network_s_eso_solved.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported netwo

In [22]:
psets.to_json(outdir / "psets.json")
pnoms.to_json(outdir / "pnoms.json")
prices.T.to_json(outdir / "prices.json")
line_loads.T.to_json(outdir / "line_loads.json")
link_loads.T.to_json(outdir / "link_loads.json")
p.T.to_json(outdir / "p.json")

In [24]:
n = pypsa.Network(root / datadir.format("2024-03-07", 4) / fn)

n.generators.to_json(outdir / "generators_static.json")
n.buses.to_json(outdir / "buses_static.json")
n.lines.to_json(outdir / "lines_static.json")
n.links.to_json(outdir / "links_static.json")
n.loads.to_json(outdir / "loads_static.json")

INFO:pypsa.io:Imported network network_s_eso_solved.nc has buses, carriers, generators, lines, links, loads


In [21]:
psets

,2024-03-07 00:00:00,2024-03-07 00:30:00,2024-03-07 01:00:00,2024-03-07 01:30:00,2024-03-07 02:00:00,2024-03-07 02:30:00,2024-03-07 03:00:00,2024-03-07 03:30:00,2024-03-07 04:00:00,2024-03-07 04:30:00,...,2024-03-07 18:30:00,2024-03-07 19:00:00,2024-03-07 19:30:00,2024-03-07 20:00:00,2024-03-07 20:30:00,2024-03-07 21:00:00,2024-03-07 21:30:00,2024-03-07 22:00:00,2024-03-07 22:30:00,2024-03-07 23:00:00
Generator,,,,,,,,,,,,,,,,,,,,,
GB0 Z10 gas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GB0 Z10 gas-fired,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GB0 Z10 interconnector,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GB0 Z11 coal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GB0 Z11 gas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GB0 Z9 gas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GB0 Z9 hydro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GB0 Z9 nuclear,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
fn = "network_s_nodal_solved.nc"
datadir = '{}_{}'
root = Path.cwd().parent / "resources" / "live_data"

n = pypsa.Network(root / datadir.format("2024-03-07", 1) / fn)

INFO:pypsa.io:Imported network network_s_nodal_solved.nc has buses, carriers, generators, lines, links, loads


In [27]:
n.generators.p_nom.sum(), n.loads.p_set.sum()

(207951.9999999976, 25993.9999999997)

In [1]:
'post_balancing_price'.split('_')[0].replace('-', '_')

'post'

In [24]:
m = n.copy()

m.generators.loc[m.generators.marginal_cost == 0., 'marginal_cost'] = 0.01
m.optimize(solver_name="gurobi")

c:\Users\lukas\miniforge3\envs\pypsa-eur\lib\site-packages\linopy\expressions.py:176: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'Generator' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates in the future (i.e., one coordinate for each multi-index level + one dimension coordinate). If you want to keep this behavior, you need to first wrap it explicitly using `mindex_coords = xarray.Coordinates.from_pandas_multiindex(mindex_obj, 'dim')` and pass it as coordinates, e.g., `xarray.Dataset(coords=mindex_coords)`, `dataset.assign_coords(mindex_coords)` or `dataarray.assign_coords(mindex_coords)`.
  ds = self.data.assign_coords({group_dim: idx})
c:\Users\lukas\miniforge3\envs\pypsa-eur\lib\site-packages\linopy\expressions.py:176: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'Line' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\lukas\AppData\Local\Temp\linopy-problem-le1arfdk.lp
Reading time = 0.02 seconds
obj: 1551 rows, 568 columns, 3604 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12650H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1551 rows, 568 columns and 3604 nonzeros
Model fingerprint: 0x594b6822
Coefficient statistics:
  Matrix range     [5e-03, 5e+01]
  Objective range  [1e-02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 3e+04]
Presolve removed 1325 rows and 269 columns
Presolve time: 0.01s
Presolved: 226 rows, 299 columns, 1493 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.208534e+05   0.000000e+00      0s
     165    3.4939000e+02   0.000000e+00   0.000000e+0

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 568 primals, 1551 duals
Objective: 3.49e+02
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper, Link-fix-p-lower, Link-fix-p-upper, Kirchhoff-Voltage-Law were not assigned to the network.


('ok', 'optimal')

In [25]:
m.buses_t.marginal_price

Bus,8838,8013,8649,4950,4951,4952,4953,4955,4956,4958,...,8021,8578,7469,8027,8016,8019,7458,8283,8093,7908
snapshot,,,,,,,,,,,,,,,,,,,,,
2024-03-18 00:30:00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
